In [258]:
import math

import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import LdaModel
from gensim.models import CoherenceModel

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
# load all metadata

stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gorka\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gorka\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\gorka\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\gorka\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [259]:
#Cargamos los datos
#Cogemos las 100 primeras descripciones como prueba
data = pd.read_csv('justice.csv')
x_train = data['facts']
y_train1 = data['first_party_winner']
y_train2 = data['issue_area']

In [260]:
#Elimina signos de puntuacion y todo a minusculas
def sent_to_words(sentences):
    for sentence in sentences:
        yield (gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(x_train))

In [261]:
#Eliminamos las stop words
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

data_words_nostops = remove_stopwords(data_words)

In [262]:
lem = WordNetLemmatizer()
ps = PorterStemmer()

In [263]:
#Lematizamos
def lemmatization(texts):
    texts_out = []
    for text in texts:
        lemText = []
        for sent in text:
            stemmed = ps.stem(sent)
            lemText.append(lem.lemmatize(stemmed))
        texts_out.append(lemText)
    return texts_out

data_lemmatized = lemmatization(data_words_nostops)

In [264]:
#Vectorizar
#Topic modeling
id2word = corpora.Dictionary(data_lemmatized)
texts = data_lemmatized
corpus = [id2word.doc2bow(text) for text in texts]
#lda = LdaModel(corpus=corpus, id2word=id2word, num_topics=20, random_state=100, update_every=1, chunksize=100, passes=20, alpha='auto',per_word_topics=True)
#print(f"TOPICOS --> {lda.print_topics()}")

lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=20, random_state=100, passes=50, eval_every=None)
count = 0
for i in lda_model.print_topics():
    print("Topic:", count, i)
    count += 1


Topic: 0 (0, '0.035*"patent" + 0.020*"use" + 0.019*"court" + 0.017*"infring" + 0.012*"appeal" + 0.010*"district" + 0.010*"feder" + 0.009*"product" + 0.009*"copyright" + 0.009*"inc"')
Topic: 1 (1, '0.043*"m" + 0.027*"st" + 0.023*"naacp" + 0.017*"basic" + 0.017*"dr" + 0.015*"levin" + 0.010*"war" + 0.009*"zenith" + 0.009*"sprint" + 0.009*"easement"')
Topic: 2 (2, '0.085*"court" + 0.031*"appeal" + 0.028*"state" + 0.028*"petit" + 0.024*"feder" + 0.023*"file" + 0.022*"district" + 0.020*"counsel" + 0.020*"habea" + 0.018*"deni"')
Topic: 3 (3, '0.046*"court" + 0.023*"arbitr" + 0.018*"district" + 0.018*"claim" + 0.017*"carolina" + 0.016*"state" + 0.016*"north" + 0.015*"feder" + 0.014*"agreement" + 0.013*"law"')
Topic: 4 (4, '0.022*"state" + 0.018*"land" + 0.016*"act" + 0.013*"tribe" + 0.011*"court" + 0.010*"servic" + 0.010*"regul" + 0.009*"unit" + 0.009*"nation" + 0.009*"feder"')
Topic: 5 (5, '0.043*"sentenc" + 0.043*"juri" + 0.033*"death" + 0.027*"murder" + 0.027*"trial" + 0.017*"convict" + 0.0

In [265]:
#Hay que cambiar el n_topics por el numero de topics del LDA
n_topics = 20
vectorized = []
count = 0

for i in lda_model[corpus]:
    index = [0]*n_topics
    for n in i:
        index[n[0]] = n[1]
    vectorized.append(index)
vectorized = pd.DataFrame(vectorized)
print(vectorized)

            0    1         2         3         4         5         6   \
0     0.000000  0.0  0.000000  0.000000  0.000000  0.118256  0.000000   
1     0.000000  0.0  0.000000  0.000000  0.131371  0.000000  0.000000   
2     0.000000  0.0  0.072559  0.000000  0.000000  0.000000  0.000000   
3     0.000000  0.0  0.000000  0.000000  0.000000  0.040615  0.000000   
4     0.182056  0.0  0.000000  0.000000  0.000000  0.000000  0.000000   
...        ...  ...       ...       ...       ...       ...       ...   
3298  0.000000  0.0  0.049182  0.000000  0.000000  0.000000  0.000000   
3299  0.087502  0.0  0.087258  0.027243  0.000000  0.000000  0.000000   
3300  0.000000  0.0  0.000000  0.021977  0.148934  0.000000  0.010125   
3301  0.000000  0.0  0.000000  0.179831  0.000000  0.000000  0.303487   
3302  0.086962  0.0  0.000000  0.115084  0.266418  0.000000  0.000000   

            7         8         9         10        11   12        13   14  \
0     0.000000  0.124620  0.053426  0.000000 

In [266]:
from sklearn.decomposition import  PCA

print("Before:", vectorized.shape)
pca = PCA(n_components='mle', svd_solver='full')
#pca = PCA(n_components=2)
pca.fit(vectorized)
labels_PCA= pca.transform(vectorized)
print("After:", labels_PCA.shape)

Before: (3303, 20)
After: (3303, 19)


In [267]:
import math
import numpy as np
def getDist(clust1, clust2):
    dist = 0
    for i in range(len(clust1)):
        dist += math.sqrt((clust1[i] - clust2[i])**2)
    return dist

In [268]:
def getDistances(clusters):
    distances = []
    for clust1 in range(len(clusters)):
        dist_clust1 = [np.NAN] * (clust1+1)
        for clust2 in [t for t in range(len(clusters)) if t > clust1]:
            dist_clust1.append(getDist(clusters[clust1], clusters[clust2]))
        distances.append(np.array(dist_clust1))
    distances = np.array(distances)
    return distances


In [ ]:
clusters = labels_PCA.copy()
n=10
distances = getDistances(clusters)
k_clusters = [[i] for i in range(len(clusters))]

while len(distances) > n:
    ind1,ind2 = np.unravel_index(np.nanargmin(distances), distances.shape)
    delete = max(ind1,ind2)
    replace = min(ind1,ind2)
    for i in [t for t in range(len(distances)) if t != ind1 and t != ind2]:
        if replace < i:
            distances[replace][i] = np.nanmin([distances[ind1][i],distances[ind2][i],distances[i][ind1],distances[i][ind2]])

        else:
            distances[i][replace] = np.nanmin([distances[ind1][i],distances[ind2][i],distances[i][ind1],distances[i][ind2]])
    distances = np.delete(distances, delete,1)
    distances = np.delete(distances, delete,0)
    k_clusters[replace] += k_clusters[delete]
    k_clusters.remove(k_clusters[delete])

kmeans_labels= [0] * len(clusters)
i = 0
for k in k_clusters:
    for c in k:
        kmeans_labels[c] = i
    i += 1


In [ ]:
import matplotlib.pyplot as plt
samples = 30
# Dibujar los puntos en el espacio, color: cluster, etiqueta-numérica: clase
# Color del punto: cluster
sc = plt.scatter(labels_PCA[:samples,0],labels_PCA[:samples,1], cmap=plt.cm.get_cmap('nipy_spectral', 20),c=kmeans_labels[:samples])
plt.colorbar()
# Etiqueta numérica: clase
for i in range(samples):
    plt.text(labels_PCA[i,0],labels_PCA[i,1], y_train2[i])